In [1]:
## Script to split the data into a federated Dataset
## Code from the same partitiondata.py file that was submitted in Part 1
## This will just create the dataset directory in the structure that we want and load the images

import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import os



def split_data(dataset, num_clients, deviation_factor=0.1):
    """
    Splits data across clients with slight deviation
    """
    client_indices = {i: [] for i in range(num_clients)}

    # Split indices based on classes
    for class_idx in range(10):
        class_indices = np.where(np.array(dataset.targets) == class_idx)[0]
        np.random.shuffle(class_indices)

        # Calculate the number of samples
        total_samples = len(class_indices)
        avg_samples_per_client = total_samples // num_clients


        splits = []
        remaining_samples = total_samples
        for client in range(num_clients):
            # Apply deviation to the average samples
            deviation = np.random.randint(-int(avg_samples_per_client * deviation_factor),
                                          int(avg_samples_per_client * deviation_factor) + 1)
            client_sample_count = avg_samples_per_client + deviation

            if client == num_clients - 1:
                client_sample_count = remaining_samples
            else:
                client_sample_count = min(client_sample_count, remaining_samples)

            remaining_samples -= client_sample_count
            splits.append(client_sample_count)

        split_class_indices = np.split(class_indices, np.cumsum(splits[:-1]))

        for client, indices in enumerate(split_class_indices):
            client_indices[client].extend(indices)

    return client_indices



# Function to save client data as jpg
def save_client_data_as_jpg(dataset, indices, client_dir, client_id):
    client_path = os.path.join(client_dir, f'client_{client_id}')
    os.makedirs(client_path, exist_ok=True)


    for idx in indices:
        img, label = dataset[idx]
        label_folder = os.path.join(client_path, f'label_{label}')
        os.makedirs(label_folder, exist_ok=True)

        # Save the image as a .jpg
        img_path = os.path.join(label_folder, f'{idx}.jpg')

        img_pil = transforms.ToPILImage()(img)
        img_pil.save(img_path)





def main():
    transform = transforms.Compose([
                                transforms.RandomHorizontalFlip(p=0.3),
                                transforms.RandomApply([transforms.RandomRotation(10)], p=0.3),
                                transforms.RandomResizedCrop(32, scale=(0.8, 1.0)),
                                # transforms.RandomApply([transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))], p=0.2),
                                transforms.ToTensor(),
                                # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])

    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

    num_clients = 10
    client_dir = './federated_clients_data/'

    # Spliting data for training and testing
    train_indices = split_data(train_dataset, num_clients, deviation_factor=0.2)
    test_indices = split_data(test_dataset, num_clients, deviation_factor=0.2)

    # Saving the  data for each client
    for client_id in range(num_clients):
        save_client_data_as_jpg(train_dataset, train_indices[client_id], client_dir, client_id)
        save_client_data_as_jpg(test_dataset, test_indices[client_id], client_dir, client_id)

    print(f"Data has been split and saved into '{client_dir}' for {num_clients} clients.")

if __name__ == '__main__':
    main()

100%|██████████| 170M/170M [00:03<00:00, 48.3MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Data has been split and saved into './federated_clients_data/' for 10 clients.


In [2]:
### Script to poison one of the clients
import os
import random
from PIL import Image

NUM_CLASSES = 10

def flip_label_per_image(original_label):
    available_labels = list(range(NUM_CLASSES))
    available_labels.remove(original_label)
    new_label = random.choice(available_labels)  # Randomly select a new label from remaining labels
    return new_label

def poison_client_data_per_image(original_dir, poisoned_dir, num_clients_to_poison):

    os.makedirs(poisoned_dir, exist_ok=True)


    client_dirs = [d for d in os.listdir(original_dir) if d.startswith('client_')]

    # Selecting client to poison
    random.shuffle(client_dirs)
    clients_to_poison = client_dirs[:num_clients_to_poison]

    print(f"Poisoning the following clients: {clients_to_poison}")

    for client in clients_to_poison:

        client_path = os.path.join(original_dir, client)


        poisoned_client_path = os.path.join(poisoned_dir, f'poisoned_{client}')
        os.makedirs(poisoned_client_path, exist_ok=True)

        for label_folder in os.listdir(client_path):
            original_label_path = os.path.join(client_path, label_folder)

            if os.path.isdir(original_label_path):
                original_label = int(label_folder.split("_")[1])

                for img_file in os.listdir(original_label_path):
                    img_path = os.path.join(original_label_path, img_file)

                    # Determine the new flipped label for this image
                    new_label = flip_label_per_image(original_label)

                    poisoned_label_path = os.path.join(poisoned_client_path, f'label_{new_label}')
                    os.makedirs(poisoned_label_path, exist_ok=True)

                    img = Image.open(img_path)
                    img.save(os.path.join(poisoned_label_path, img_file))

    print(f"Poisoning complete. Poisoned data saved in: {poisoned_dir}")


original_data_dir = './federated_clients_data'
poisoned_data_dir = './poisoned_clients_data'   # new directory to save poisoned data
num_clients_to_poison = 1  # No of cleints to poison


poison_client_data_per_image(original_data_dir, poisoned_data_dir, num_clients_to_poison)

Poisoning the following clients: ['client_7']
Poisoning complete. Poisoned data saved in: ./poisoned_clients_data


In [34]:
# These commands will move the poisoned client data to the original directory so that we have one poisoned client
!cd federated_clients_data/
!rm -r client_2
!mv poisoned_client_2 client_2
# Move the directory to the ./federated_clients_data

rm: cannot remove 'client_2': No such file or directory
mv: cannot stat 'poisoned_client_2': No such file or directory


In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [4]:
print(torch.__version__)

2.5.0+cu121


In [5]:
!pip install -q flwr[simulation] flwr-datasets[vision]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 478.6/478.6 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cu

In [6]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()

Training on cpu
Flower 1.12.0 / PyTorch 2.5.0+cu121


In [7]:
NUM_CLIENTS = 10
BATCH_SIZE = 32

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
import os
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image

In [9]:
## Custom Dataset class that reads the data for the client from the appropriate directory based on the client ID provided
class ClientDataset(Dataset):
  def __init__(self,root_dir,client_id,transform=None):
    self.client_dir = os.path.join(root_dir,client_id)
    self.transform = transform
    self.images = []
    self.labels = []
    self._load_data()
  def _load_data(self):
    for label in os.listdir(self.client_dir):
            label_dir = os.path.join(self.client_dir, label)
            if os.path.isdir(label_dir):
                for img_file in os.listdir(label_dir):
                    img_path = os.path.join(label_dir, img_file)
                    self.images.append(img_path)
                    self.labels.append(int(label[-1]))

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_path = self.images[idx]
    image = Image.open(img_path)
    label = self.labels[idx]
    if self.transform:
        image = self.transform(image)
    return image, label


In [10]:
## Function that is called by a Flower Framework Client to load the dataset
## Uses the Client Dataset class created above to using the specified client id derived from the partition id
def load_datasets(partition_id:int):
    root_dir = "/content/federated_clients_data"
    client_id = "client_" + str(partition_id)
    transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to match CIFAR-10 dimensions
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    client_dataset = ClientDataset(root_dir=root_dir, client_id=client_id, transform=transform)

    train_size = int(0.8 * len(client_dataset))
    test_size = len(client_dataset) - train_size

    train_dataset, test_dataset = random_split(client_dataset, [train_size, test_size])

    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset,[train_size,val_size])
    trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valloader = DataLoader(val_dataset,batch_size=BATCH_SIZE,shuffle=False)
    testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    return trainloader,valloader,testloader




In [11]:
## Defines the neural network architecture that would be used by all the clients
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 32x32x3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # convolutional layer (sees 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 8x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, 10)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(-1, 64 * 4 * 4)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x

In [12]:
import torch.optim as optim

In [13]:
# Train and Test functions that are used by clients fit and evaluate methods
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for batch in trainloader:
            images, labels = batch
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item()*labels.size(0)
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch
            outputs = net(images)
            loss += criterion(outputs, labels).item()*labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total

    return loss, accuracy

In [14]:
# Functions to get and set parameters for the model on clients
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [15]:
# Global dictionaries to track the metrics across rounds for each client
accuracy_per_client_per_round = {}
loss_per_client_per_round = {}

In [16]:
# Defines a flower client class
class FlowerClient(NumPyClient):
    def __init__(self, net, trainloader, valloader,partitionId):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.id = partitionId
        self.metrics_list = []

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {"partition":str(self.id)}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        self.metrics_list.append((loss,accuracy))
        return float(loss), len(self.valloader), {"accuracy": float(accuracy),"loss": float(loss),"partition":str(self.id)}

In [17]:
## A global cache to store the clients
clients_cache = {}

In [18]:
# Function to create clients that will passed to the simulation so that the server can create clients as required
def client_fn(context: Context) -> Client:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data partition
    # Read the node_config to fetch data partition associated to this node
    partition_id = context.node_config["partition-id"]
    if partition_id not in clients_cache:
      trainloader, valloader, _ = load_datasets(partition_id=partition_id)
      clients_cache[partition_id] = FlowerClient(net, trainloader, valloader,partition_id).to_client()

    # Create a single Flower client representing a single organization
    # FlowerClient is a subclass of NumPyClient, so we need to call .to_client()
    # to convert it to a subclass of `flwr.client.Client`
    return clients_cache[partition_id]


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [19]:
# Global dictionary to store Reputation and Trust for each client
reputation_per_round = {}
trust_per_round = {}
trust_threshold = 0.5
old_parameters = None
excluded_client_partitions = []

In [20]:
import numpy as np

def convert_parameters_to_numpy_array(parameter_object):
  numpy_weights = []
  # Iterate through each byte string in tensors

  for tensor in parameter_object.tensors:
      # Convert byte string to numpy array
      np_array = np.frombuffer(tensor, dtype=np.float64)
      numpy_weights.append(np_array)
  numpy_weights = np.concatenate(numpy_weights)
  numpy_weights.flatten()
  return numpy_weights

In [21]:
def compute_normalized_L2_distance_between_parameters(global_weights, client_weights):
    if global_weights is None or client_weights is None:
        return 0

    global_weights = convert_parameters_to_numpy_array(global_weights)
    client_weights = convert_parameters_to_numpy_array(client_weights)

    # Calculate the L2 distance between global and client weights
    l2_distance = np.linalg.norm(np.subtract(global_weights, client_weights))
    print(f"L2 Distance is {l2_distance}")

    return l2_distance

In [22]:
import math
def compute_reputation(prev_reputation,threshold,epochvalue,d):
  reputation = 0
  if epochvalue == 2:
    # initial reputation
    reputation = 1 - d
  else:
    if d == 0:
      return 1
    if d < threshold:
      reputation = (prev_reputation + (1/d)) - (prev_reputation/epochvalue)
    if d >= threshold:
      reputation = (prev_reputation + (1/d)) - math.exp(-(1-(d * (prev_reputation/epochvalue))))
    if reputation < 0:
      reputation = 0
    if reputation > 1:
      reputation = 1
  return reputation

In [23]:
import math

In [24]:
def compute_trust(reputation,d):
  trust = math.sqrt((reputation)**2 + (d)**2) - math.sqrt((1-reputation)**2 + (1-d)**2)
  if trust >= 1:
    trust = 1
  elif trust <=0:
    trust = 0
  return trust

In [25]:
def exclude_client():
  pass

In [26]:
def parameters_to_ndarrays(parameters):
    """Convert Flower Parameters object to a list of NumPy arrays."""
    return [np.array(tensor, dtype=np.float32) for tensor in parameters.tensors]

In [27]:
##perform aggregation based on FedAvg
## distribute aggregated global model beach to clients
import pickle

In [29]:
# Custom Strategy Class to store metrics for each round into the global dictonaries
from flwr.server.strategy import FedAvg

class CustomStrategy(FedAvg):

  def aggregate_fit(self, rnd, results, failures):
    # Access each client's parameters and metrics in the results list
        client_parameters = []
        trusted_clients = []
        updated_reputations = {}
        global old_parameters
        global reputation_per_round
        global trust_per_round
        global trust_threshold
        global excluded_client_partitions
        global_parameters = None
        if old_parameters is None:
            old_parameters = self.initial_parameters
            global_parameters = old_parameters
        else:
            global_parameters = old_parameters
        global_aggregated_parameters = super().aggregate_fit(rnd, results, failures)[0]


        if rnd <=1:
          aggregated_parameters = super().aggregate_fit(rnd, results, failures)
          old_parameters = aggregated_parameters[0]
          return aggregated_parameters
        elif rnd >=2:
          distances = []
          for client_id, (client_proxy, fitres_object) in enumerate(results):
              client_identifier = client_proxy.cid
              client_parameters_current = fitres_object.parameters

              # Store the client parameters for further processing
              client_parameters.append(client_parameters_current)


              # Example of using metrics: access accuracy or loss, if needed
              loss = fitres_object.metrics.get("loss")
              accuracy = fitres_object.metrics.get("accuracy")
              partition_id = fitres_object.metrics.get("partition")

              if rnd == 2:
                reputation_per_round[partition_id] = None
                trust_per_round[partition_id] = None
              # Calculation L2 Distance
              print(f"isNone Global {global_parameters is None}   Client{client_parameters_current is None}")
              d = compute_normalized_L2_distance_between_parameters(global_aggregated_parameters,client_parameters_current)
              distances.append(d)
              print(f"L2 Distance is {d}")
              # Figure out the value for threshold - Currently 0.5
              # current_reputation = compute_reputation(reputation_per_round[partition_id],0.5,rnd,d)
              # reputation_per_round[partition_id] = current_reputation
              # print(f"Reputation for Client {partition_id} is {current_reputation}")
              # current_trust = compute_trust(current_reputation,d)
              # trust_per_round[partition_id] = current_trust
              # print(f"Trust for Client {partition_id} is {current_trust}")

              # if rnd > 10:
              #   # Check if the updated reputation meets the threshold
              #   if current_trust >= trust_threshold:
              #       trusted_clients.append((parameters, num_examples))
              # else:
              #   trusted_clients.append((parameters, num_examples))
          max_dist = max(distances)
          normalized_distances = [d / max_dist for d in distances]
          for client_id, (client_proxy, fitres_object) in enumerate(results):
              client_identifier = client_proxy.cid
              partition_id = fitres_object.metrics.get("partition")
              current_reputation = compute_reputation(reputation_per_round[partition_id],0.2,rnd,normalized_distances[client_id])
              reputation_per_round[partition_id] = current_reputation
              print(f"Reputation for Client {partition_id} is {current_reputation}")
              current_trust = compute_trust(current_reputation,d)
              trust_per_round[partition_id] = current_trust
              print(f"Trust for Client {partition_id} is {current_trust}")
              if rnd > 30:
                if current_trust < trust_threshold:
                  excluded_client_partitions.append(partition_id)
                  print(f"Excluding Client {partition_id} from Round {rnd}")
          if rnd > 30:
            result_trusted = []
            for client_id, (client_proxy, fitres_object) in enumerate(results):
                client_identifier = client_proxy.cid
                partition_id = fitres_object.metrics.get("partition")
                if partition_id in excluded_client_partitions:
                  continue
                result_trusted.append((client_proxy,fitres_object))
            aggregated_parameters = super().aggregate_fit(rnd, result_trusted, failures)
            old_parameters = aggregated_parameters[0]
            return aggregated_parameters
          else:
            aggregated_parameters = super().aggregate_fit(rnd, results, failures)
            old_parameters = aggregated_parameters[0]
            return aggregated_parameters

  def aggregate_evaluate(self,rnd,results,failure):
    if failure:
      print(f"Round {rnd} had {len(failure)} failures")
    accuracy_list = [r.metrics["accuracy"] for _, r in results]

    for _,r in results:
      if r.metrics["partition"] not in accuracy_per_client_per_round:
        accuracy_per_client_per_round[r.metrics["partition"]] = []
      accuracy_per_client_per_round[r.metrics["partition"]].append((rnd,r.metrics["accuracy"]))
      if r.metrics["partition"] not in loss_per_client_per_round:
        loss_per_client_per_round[r.metrics["partition"]] =[]
      loss_per_client_per_round[r.metrics["partition"]].append((rnd,r.metrics["loss"]))

    average_accuracy = sum(accuracy_list) / len(accuracy_list)
    print(f"Round {rnd} accuracy: {average_accuracy}")
    print(f"Accuracy List for Round {rnd} : {accuracy_list}")


    return super().aggregate_evaluate(rnd, results, failure)

In [30]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}

In [31]:
# Global list for storing aggreagated metric
aggregate_metric_list = []

In [32]:
# Function that calculates the average loss and accuracy
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    losses = [num_examples * m["loss"] for num_examples, m in metrics]  # For weighted loss
    examples = [num_examples for num_examples, _ in metrics]
    accuracy = sum(accuracies) / sum(examples)
    loss = sum(losses) / sum(examples)
    aggregate_metric_list.append({"accuracy": accuracy, "loss": loss})
    # Aggregate and return custom metric (weighted average)
    return {"accuracy": accuracy, "loss": loss}

In [33]:
# Function that creates the Server object contains the Custom Stategy mentioned above
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.
    """

    strategy = CustomStrategy(
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=10,
        min_evaluate_clients=10,
        min_available_clients=10,
        evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
    )

    # Configure the server for 100 rounds of training
    config = ServerConfig(num_rounds=200)

    return ServerAppComponents(strategy=strategy, config=config)

In [34]:
# Running the simulation
import os
os.environ["RAY_DEDUP_LOGS"] = "0"

# Create a new server instance with the updated FedAvg strategy
server = ServerApp(server_fn=server_fn)

# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

print(accuracy_per_client_per_round)
print(loss_per_client_per_round)
print(aggregate_metric_list)

DEBUG:flwr:Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=200, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=1944) 2024-11-04 10:56:55.949158: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=1944) 2024-11-04 10:56:55.996052: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=1944) 2024-11-04 10:56:56.016610: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=1945) 2024-11-04 10:56:58.356815: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

Round 1 accuracy: 0.1019645089414745
Accuracy List for Round 1 : [0.10640394088669951, 0.11491712707182321, 0.09884332281808622, 0.10650224215246637, 0.09642470205850487, 0.09331919406150584, 0.10743801652892562, 0.09205020920502092, 0.1288604898828541, 0.07488584474885844]


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)


isNone Global False   ClientFalse
L2 Distance is 1.6429889425788859e-10
L2 Distance is 1.6429889425788859e-10
isNone Global False   ClientFalse
L2 Distance is 6.328963496547094e-10
L2 Distance is 6.328963496547094e-10
isNone Global False   ClientFalse
L2 Distance is 2.1110282927496755e-10
L2 Distance is 2.1110282927496755e-10
isNone Global False   ClientFalse
L2 Distance is 2.176875207491316e-10
L2 Distance is 2.176875207491316e-10
isNone Global False   ClientFalse
L2 Distance is 2.3943955499342255e-10
L2 Distance is 2.3943955499342255e-10
isNone Global False   ClientFalse
L2 Distance is 2.8624599750887707e-10
L2 Distance is 2.8624599750887707e-10
isNone Global False   ClientFalse
L2 Distance is 1.713062490716213e-10
L2 Distance is 1.713062490716213e-10
isNone Global False   ClientFalse
L2 Distance is 3.020338238042016e-10
L2 Distance is 3.020338238042016e-10
isNone Global False   ClientFalse
L2 Distance is 2.894487038983983e-10
L2 Distance is 2.894487038983983e-10
isNone Global False 

INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [ROUND 3]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


...

isNone Global False   ClientFalse
L2 Distance is 4.464221527552396e-05
L2 Distance is 4.464221527552396e-05
isNone Global False   ClientFalse
L2 Distance is 7.024486423068921e-06
L2 Distance is 7.024486423068921e-06
isNone Global False   ClientFalse
L2 Distance is 8.31169918230559e-06
L2 Distance is 8.31169918230559e-06
isNone Global False   ClientFalse
L2 Distance is 4.32924209986277e-06
L2 Distance is 4.32924209986277e-06
isNone Global False   ClientFalse
L2 Distance is 1.1440615358223123e-05
L2 Distance is 1.1440615358223123e-05
isNone Global False   ClientFalse
L2 Distance is 1.176361396907421e-05
L2 Distance is 1.176361396907421e-05
isNone Global False   ClientFalse
L2 Distance is 9.727729449365501e-06
L2 Distance is 9.727729449365501e-06
isNone Global False   ClientFalse
L2 Distance is 5.896035197251653e-06
L2 Distance is 5.896035197251653e-06
isNone Global False   ClientFalse
L2 Distance is 2.7000700371730096e-05
L2 Distance is 2.7000700371730096e-05
isNone Global False   Client

INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 200 round(s) in 12968.60s
INFO :      	History (loss, distributed):
INFO :      		round 1: 2.2994471531311236
INFO :      		round 2: 2.2960580581169645
INFO :      		round 3: 2.290970456236828
INFO :      		round 4: 2.2818530517702236
INFO :      		round 5: 2.262024872615345
INFO :      		round 6: 2.217333234856168
INFO :      		round 7: 2.150667216729186
INFO :      		round 8: 2.107222787470437
INFO :      		round 9: 2.0895977138902753
INFO :      		round 10: 2.051729401226883
INFO :      		round 11: 2.037547838022284
INFO :      		round 12: 2.021614192902198
INFO :      		round 13: 1.996958776478196
INFO :      		round 14: 1.9920347239975087
INFO :      		round 15: 1.9703249916515566
INFO :      		round 16: 1.9626632677130254
INFO :      		round 17: 1.9365800724001503
INFO :      		round 18: 1.913793329099628
INFO :      		round 19: 1.8995229436279326
INFO : 

Round 200 accuracy: 0.666033590784257
Accuracy List for Round 200 : [0.7305644302449414, 0.7207112970711297, 0.7576354679802956, 0.7202944269190326, 0.7399103139013453, 0.7367280606717227, 0.7465535524920467, 0.7579908675799086, 0.030939226519337018, 0.71900826446281]


INFO :      		round 137: 1.3634180434176322
INFO :      		round 138: 1.3643923396321351
INFO :      		round 139: 1.3255839986248128
INFO :      		round 140: 1.3578509097829126
INFO :      		round 141: 1.3398135821809598
INFO :      		round 142: 1.340501859780448
INFO :      		round 143: 1.3428135443678924
INFO :      		round 144: 1.3307247797485702
INFO :      		round 145: 1.3071809080257457
INFO :      		round 146: 1.3428100752691292
INFO :      		round 147: 1.3452149933781172
INFO :      		round 148: 1.335354493902968
INFO :      		round 149: 1.3205120733231945
INFO :      		round 150: 1.3038407793806437
INFO :      		round 151: 1.3192764277520215
INFO :      		round 152: 1.333522182892954
INFO :      		round 153: 1.338318580963782
INFO :      		round 154: 1.3062430304089299
INFO :      		round 155: 1.320716950209187
INFO :      		round 156: 1.3129389329121028
INFO :      		round 157: 1.3160427841970257
INFO :      		round 158: 1.3087900817396398
INFO :      		round 159: 1.2908526145

{'5': [(1, 0.10640394088669951), (2, 0.13596059113300493), (3, 0.14679802955665025), (4, 0.19507389162561575), (5, 0.20886699507389161), (6, 0.24433497536945814), (7, 0.24334975369458128), (8, 0.2817733990147783), (9, 0.2413793103448276), (10, 0.2630541871921182), (11, 0.25320197044334974), (12, 0.30640394088669953), (13, 0.3024630541871921), (14, 0.31231527093596056), (15, 0.3024630541871921), (16, 0.3044334975369458), (17, 0.3359605911330049), (18, 0.35073891625615766), (19, 0.32610837438423645), (20, 0.3556650246305419), (21, 0.33201970443349754), (22, 0.3665024630541872), (23, 0.35763546798029555), (24, 0.39408866995073893), (25, 0.38226600985221676), (26, 0.367487684729064), (27, 0.4147783251231527), (28, 0.39113300492610836), (29, 0.412807881773399), (30, 0.4068965517241379), (31, 0.41083743842364534), (32, 0.4295566502463054), (33, 0.4463054187192118), (34, 0.43842364532019706), (35, 0.4088669950738916), (36, 0.4315270935960591), (37, 0.4433497536945813), (38, 0.4492610837438424

In [35]:
import pickle

In [36]:
# Saving the metric files to disk using pickle
with open('poisoned_accuracy_per_client_per_round.pkl', 'wb') as f:
    pickle.dump(accuracy_per_client_per_round, f)
with open('poisoned_loss_per_client_per_round.pkl','wb') as f:
    pickle.dump(loss_per_client_per_round,f)
with open('poisoned_aggregate_metric_list.pkl','wb') as f:
    pickle.dump(aggregate_metric_list,f)

In [ ]:
# Client 7 was poisoned in this case